In [2]:
#!/usr/bin/env python3
import paratext
import pandas
import lz4.frame
import gzip
import io
import pyarrow.parquet as pq
import pyarrow as pa
import numpy as np
import datetime
import matplotlib.pyplot as plt

from glob import glob
from plumbum.cmd import rm

In [3]:
def load_data(filename, filter_initial=True):
  df = pq.read_table(filename).to_pandas()
  if filter_initial:
    df = df[df['Event Type'] != 'Initial']
  return df

def get_second_data(df, current_second):
  time = sec2string(current_second)
  return df.loc[df['Event Time'].values == time]

def get_minute_data(df, current_minute):
  time = min2string(current_minute)
  next_time = min2string(current_minute + 1)
  return df.loc[(df['Event Time'].values >= time) & (df['Event Time'].values < next_time)]

In [4]:
def sec2string(sec):
  m, s = divmod(sec, 60)
  h, m = divmod(m, 60)
  return "%02d:%02d:%02d" %(h, m, s)

def min2string(minute):
  h, m = divmod(minute, 60)
  return "%02d:%02d:00" %(h, m)

In [5]:
def get_avg_price(df_chunk, percent_change, prev_price, when):
  df_chunk = filter_df(df_chunk, event_type='Fill')
  if len(df_chunk) == 0:
    current_avg_price = prev_price
  else:
    if when == 'start':
      current_avg_price = df_chunk.iloc[0, -1]
    elif when == 'end':
      current_avg_price = df_chunk.iloc[-1, -1]
  return current_avg_price

def calc_percent_change(current_price, prev_price):
  try:
    percent_change = (current_price - prev_price) / prev_price
  except:
    percent_change = 0.0
  return percent_change

In [6]:
def filter_df(df_chunk, side=None, event_type=None, order_type=None):
  if side is not None:
    df_chunk = df_chunk.loc[df_chunk['Side'].values == side]
  if event_type is not None:
    df_chunk = df_chunk.loc[df_chunk['Event Type'].values == event_type]
  if order_type is not None:
    df_chunk = df_chunk.loc[df_chunk['Order Type'].values == order_type]
  return df_chunk 

def get_frequency(df_chunk):
  return len(df_chunk)
             
def get_volume(df_chunk, volume_type=None):
  if volume_type=='filled':
    return sum(df_chunk['Fill Price (USD)'] * df_chunk['Fill Quantity (BTC)'])
  if volume_type=='unfilled':
    return sum(df_chunk['Limit Price (USD)'] * df_chunk['Original Quantity (BTC)'])

def calculate_percentage(value1, value2):
  if value1 == 0.0 and value2 == 0.0:
    percentage = 0.5
  else:
    try:
      percentage = value1 / (value1 + value2 + 1e-6)
    except:
      percentage = None
  return percentage

In [7]:
def one_hot(index, length):
  onehot = [0.]*length
  onehot[index] = 1.
  return onehot
    
def extract_temporal_features(df_chunk):
  year, month, day = df_chunk['Event Date'].values[0].split('-')
  day_of_week = int(datetime.datetime(int(year), int(month), int(day)).weekday())
  hour = int(df_chunk['Event Time'].values[0][0:2])
  month = int(month) - 1
  return one_hot(month, 12), one_hot(day_of_week, 7), one_hot(hour, 24)

In [8]:
def vol_freq(df_chunk, volume_type):
  return get_volume(df_chunk, volume_type=volume_type), get_frequency(df_chunk)

def get_raw_features(df_chunk, side=None):
  x = {}
  x['vol_markets'], x['freq_markets']                   = vol_freq(filter_df(df_chunk, side=side, event_type='Fill', order_type='market'), volume_type='filled')
  x['vol_filled_limits'], x['freq_filled_limits']       = vol_freq(filter_df(df_chunk, side=side, event_type='Fill', order_type='limit'), volume_type='filled')
  x['vol_placed_limits'], x['freq_placed_limits']       = vol_freq(filter_df(df_chunk, side=side, event_type='Place', order_type='limit'), volume_type='unfilled')
  x['vol_cancelled_limits'], x['freq_cancelled_limits'] = vol_freq(filter_df(df_chunk, side=side, event_type='Cancel', order_type='limit'), volume_type='unfilled')  
  return x 

def compute_features(x_buy, x_sell):
  # Buys:
  # -Volume of Filled Markets vs Filled Limits
  # -Volume of Placed Limits vs Filled Limits
  # -Frequency of Filled Markets vs Filled Limits
  # -Frequency of Placed Limits vs Filled Limits

  # Sells:
  # -Volume of Filled Markets vs Filled Limits
  # -Volume of Placed Limited vs Filled Limits
  # -Frequency of Filled Markets vs Filled Limits
  # -Frequency of Placed Limits vs Filled Limits

  # Buys vs Sells:
  # -Volume of Filled Market Sells vs Volume of Filled Market Buys
  # -Volume of Placed Limit Sells vs Volume of Placed Limit Buys
  # -Volume of Cancelled Limit Sells vs Volume Cancelled Limit Buys
  # -Frequency of Filled Market Sells vs Frequency of Filled Market Buys
  # -Frequency of Placed Limit Sells vs Frequency of Placed Limit Buys
  # -Frequency of Cancelled Limit Sells vs Frequency of Cancelled Limit Buys
  
  features = []
  # Buys:
  features.append(calculate_percentage(x_buy['vol_markets'],        x_buy['vol_filled_limits']))
  features.append(calculate_percentage(x_buy['vol_placed_limits'],  x_buy['vol_filled_limits']))
  features.append(calculate_percentage(x_buy['freq_markets'],       x_buy['freq_filled_limits']))
  features.append(calculate_percentage(x_buy['freq_placed_limits'], x_buy['freq_filled_limits']))

  # Sells:
  features.append(calculate_percentage(x_sell['vol_markets'],        x_sell['vol_filled_limits']))
  features.append(calculate_percentage(x_sell['vol_placed_limits'],  x_sell['vol_filled_limits']))
  features.append(calculate_percentage(x_sell['freq_markets'],       x_sell['freq_filled_limits']))
  features.append(calculate_percentage(x_sell['freq_placed_limits'], x_sell['freq_filled_limits']))

  # Buys vs Sells:
  features.append(calculate_percentage(x_sell['vol_markets'],           x_buy['vol_markets']))
  features.append(calculate_percentage(x_sell['vol_placed_limits'],     x_buy['vol_placed_limits']))
  features.append(calculate_percentage(x_sell['vol_cancelled_limits'],  x_buy['vol_cancelled_limits']))
  features.append(calculate_percentage(x_sell['freq_markets'],          x_buy['freq_markets']))
  features.append(calculate_percentage(x_sell['freq_placed_limits'],    x_buy['freq_placed_limits']))
  features.append(calculate_percentage(x_sell['freq_cancelled_limits'], x_buy['freq_cancelled_limits']))

  return features

In [9]:
def get_all_features(df_chunk, percent_change, prev_price):
  
  # Current price, percent change
  current_price = get_avg_price(df_chunk, percent_change, prev_price, when='end')
  percent_change = calc_percent_change(current_price, prev_price)
  
  feature_vec = [current_price, percent_change]
  
  # Order book features
  x_buy  = get_raw_features(df_chunk, side='buy')
  x_sell = get_raw_features(df_chunk, side='sell')
  feature_vec.extend(compute_features(x_buy, x_sell))
  
  # Temporal features
  month_vec, day_vec, hour_vec = extract_temporal_features(df_chunk)
  feature_vec.extend(month_vec)
  feature_vec.extend(day_vec)
  feature_vec.extend(hour_vec)
  
  return feature_vec

In [10]:
def write_tmp_parquet(df, outfile):
  outfile = outfile.replace('cboe/parquet_BTCUSD/', 'cboe/parquet_preprocessed_BTCUSD/')
  pq.write_table(pa.Table.from_pandas(df), outfile, compression='snappy')

In [11]:
def preprocess_day(filename, visualize=True, write_parquet=False, verbose=True):
  print(filename)
  df = load_data(filename, filter_initial=True)
  
  # Initialize previous price
  percent_change=0.0
  prev_price = get_avg_price(df, percent_change=None, prev_price=None, when='start')
  
  # Compute feature vector for each minute of the day
  all_X = []
  for minute in range(24*60):
    if verbose:
      if minute%100 == 0:
        print('Minutes:', minute)

    # Select one minute of data from order book
    df_chunk = get_minute_data(df, minute)
    if len(df_chunk) == 0: # skip minutes with no data
      continue
    
    # Extract features, X
    X = get_all_features(df_chunk, percent_change, prev_price)
    prev_price = X[0]
    percent_change = X[1]
    #all_X.append(X[1:])
    all_X.append(X)

  #columns = ['current_price','percent_change',
  columns = ['current_price', 'percent_change',
             'buy_vol_mark_vs_fillLim','buy_vol_placeLim_vs_fillLim','buy_freq_mark_vs_fillLim','buy_freq_placeLim_vs_fillLim',
             'sell_vol_mark_vs_fillLim','sell_vol_placeLim_vs_fillLim','sell_freq_mark_vs_fillLim','sell_freq_placeLim_vs_fillLim',
             'vol_markSells_vs_markBuys','vol_placeLimSells_vs_placeLimBuys','vol_CancelLimSells_vs_CancelLimBuys',
             'freq_markSells_vs_markBuys','freq_placeLimSells_vs_placeLimBuys','freq_CancelLimSells_vs_CancelLimBuys',
             'm0','m1','m2','m3','m4','m5','m6','m7','m8','m9','m10','m11',
             'd0','d1','d2','d3','d4','d5','d6',
             'h0','h1','h2','h3','h4','h5','h6','h7','h8','h9','h10','h11',
             'h12','h13','h14','h15','h16','h17','h18','h19','h20','h21','h22','h23']
  # Convert to pandas DF
  new_df = pandas.DataFrame.from_records(all_X, columns=columns) 
  
  # Compute labels, Y
  new_df = calculate_y(new_df)
  
  # Write DF to tmp file to later be concatenated with all others
  if write_parquet:
    write_tmp_parquet(new_df, filename)

  # Visualize
  if visualize:
    visualize_features(new_df)

  return new_df

In [12]:
def check_number_of_events(df, timesteps, resolution='minute', event_type=None, order_type=None):
  
  # Filter data
  if event_type is not None:
    df = df.loc[df['Event Type'].values == event_type]
  if order_type is not None:
    df = df.loc[df['Order Type'].values == order_type]
  
  chunk_lengths = []
  # Minute resolution
  if resolution == 'minute':
    for minute in range(timesteps):
      chunk_lengths.append(len(get_minute_data(df, minute)))
      if minute%200 == 0:
        print('Minute:', minute)
        
  # Second resolution
  elif resolution == 'second':
    for sec in range(timesteps):
      chunk_lengths.append(len(get_second_data(df, sec)))
      if sec%100 == 0:
        print('Second', sec)

  # Visualize
  plt.figure(figsize=(20,2));
  plt.plot(chunk_lengths);
  plt.figure(figsize=(20,5));
  plt.hist(chunk_lengths, bins=40);
  
  return chunk_lengths

In [13]:
def calculate_y(new_df):
  new_df['y_percent_change'] = new_df.iloc[:,1]
  new_df['y_percent_change'] = new_df['y_percent_change'].shift(-1)
  new_df = new_df[:-1]
  return new_df

In [14]:
def visualize_features(df):
  for column_idx in range(16):
    title = df.columns[column_idx]
    df.plot(y = column_idx, figsize=(20,2), title=title)

In [15]:
# # Visualize events
# event_type = "Fill"
# order_type = None
# resolution = 'minute'
# timesteps = 24*60

# chunk_lengths = check_number_of_events(df, timesteps=timesteps, resolution=resolution,
#                                        event_type=event_type, order_type=order_type)

In [19]:
# # Preprocess all files
count = 0
filenames = sorted(glob('cboe/parquet_BTCUSD/*.parquet'))
#filenames.reverse()
for day in range(len(filenames)):
  filename = filenames[day]
  new_df = preprocess_day(filename, write_parquet=True, visualize=False, verbose=False)
  count += 1
  print(count, '/', len(filenames))

cboe/parquet_BTCUSD/BTCUSD_order_book_20151008.parquet
1 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20151009.parquet
2 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20151010.parquet
3 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20151011.parquet
4 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20151012.parquet
5 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20151013.parquet
6 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20151014.parquet
7 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20151015.parquet
8 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20151016.parquet
9 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20151017.parquet
10 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20151018.parquet
11 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20151019.parquet
12 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20151020.parquet
13 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20151021.parquet
14 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20151022.parquet
15 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20151023.pa

128 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20160213.parquet
129 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20160214.parquet
130 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20160215.parquet
131 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20160216.parquet
132 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20160217.parquet
133 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20160218.parquet
134 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20160219.parquet
135 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20160220.parquet
136 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20160221.parquet
137 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20160222.parquet
138 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20160223.parquet
139 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20160224.parquet
140 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20160225.parquet
141 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20160226.parquet
142 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20160227.parquet
143 / 867
cboe/parquet_BT

255 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20160619.parquet
256 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20160620.parquet
257 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20160621.parquet
258 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20160622.parquet
259 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20160623.parquet
260 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20160624.parquet
261 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20160625.parquet
262 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20160626.parquet
263 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20160627.parquet
264 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20160628.parquet
265 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20160629.parquet
266 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20160630.parquet
267 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20160701.parquet
268 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20160702.parquet
269 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20160703.parquet
270 / 867
cboe/parquet_BT

382 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20161024.parquet
383 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20161025.parquet
384 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20161026.parquet
385 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20161027.parquet
386 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20161028.parquet
387 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20161029.parquet
388 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20161030.parquet
389 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20161031.parquet
390 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20161101.parquet
391 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20161102.parquet
392 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20161103.parquet
393 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20161104.parquet
394 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20161105.parquet
395 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20161106.parquet
396 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20161107.parquet
397 / 867
cboe/parquet_BT

509 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20170228.parquet
510 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20170301.parquet
511 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20170302.parquet
512 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20170303.parquet
513 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20170304.parquet
514 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20170305.parquet
515 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20170306.parquet
516 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20170307.parquet
517 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20170308.parquet
518 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20170309.parquet
519 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20170310.parquet
520 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20170311.parquet
521 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20170312.parquet
522 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20170313.parquet
523 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20170314.parquet
524 / 867
cboe/parquet_BT

636 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20170705.parquet
637 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20170706.parquet
638 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20170707.parquet
639 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20170708.parquet
640 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20170709.parquet
641 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20170710.parquet
642 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20170711.parquet
643 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20170712.parquet
644 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20170713.parquet
645 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20170714.parquet
646 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20170715.parquet
647 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20170716.parquet
648 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20170717.parquet
649 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20170718.parquet
650 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20170719.parquet
651 / 867
cboe/parquet_BT

763 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20171109.parquet
764 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20171110.parquet
765 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20171111.parquet
766 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20171112.parquet
767 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20171113.parquet
768 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20171114.parquet
769 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20171115.parquet
770 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20171116.parquet
771 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20171117.parquet
772 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20171118.parquet
773 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20171119.parquet
774 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20171120.parquet
775 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20171121.parquet
776 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20171122.parquet
777 / 867
cboe/parquet_BTCUSD/BTCUSD_order_book_20171123.parquet
778 / 867
cboe/parquet_BT

,current_price,percent_change,buy_vol_mark_vs_fillLim,buy_vol_placeLim_vs_fillLim,buy_freq_mark_vs_fillLim,buy_freq_placeLim_vs_fillLim,sell_vol_mark_vs_fillLim,sell_vol_placeLim_vs_fillLim,sell_freq_mark_vs_fillLim,sell_freq_placeLim_vs_fillLim,...,h15,h16,h17,h18,h19,h20,h21,h22,h23,y_percent_change
0,242.500000,0.000000,0.5,0.500000,0.500000,0.500000,0.5,1.000000,0.500000,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,242.500000,0.000000,0.5,0.500000,0.500000,0.500000,0.5,1.000000,0.500000,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,242.500000,0.000000,0.5,0.500000,0.500000,0.500000,0.5,1.000000,0.500000,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,242.500000,0.000000,0.5,0.500000,0.500000,0.500000,0.5,1.000000,0.500000,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,242.500000,0.000000,0.5,0.500000,0.500000,0.500000,0.5,1.000000,0.500000,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
5,242.500000,0.000000,0.5,0.500000,0.500000,0.500000,0.5,1.000000,0.500000,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
6,242.500000,0.000000,0.5,0.500000,0.500000,0.500000,0.5,1.000000,0.500000,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
7,242.500000,0.000000,0.5,0.500000,0.500000,0.500000,0.5,1.000000,0.500000,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
8,242.500000,0.000000,0.5,0.500000,0.500000,0.500000,0.5,0.999922,0.500000,0.999999,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
9,242.500000,0.000000,0.5,1.000000,0.500000,1.000000,0.5,0.500000,0.500000,0.500000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
